<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1b70627412903baed48cb9c4f0ac774f60b841b7c8bb82473b45d76f985161ad
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-20 14:26:21
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.16 L
Current:  1.45 C
-------------------
Today PnL: 3.92 K (0.03%)
Current PnL: -21.08 L (-13.81%)
CY Booked + Current PnL: -7.12 L (-4.66%)
-------------------
Total profit:  1.82 L
Total loss:  -22.89 L
-------------------
Total Booked + Current PnL: 19.86 L (15.81%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.65%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.99 L (59.42%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.13%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.77,-7.40,8.01,0.02,12947.0,-12919.0,161640.0,146.65,77.0,M-SC,3.05,234.0,-1.00,1.13,40.60,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.50,-14.51,17.08,0.09,14715.0,-14625.0,86152.0,100.00,48.0,M-SC,4.01,253.0,-0.99,0.60,13.18,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.80,10.54,8.28,19.69,17549.0,20208.0,211950.0,-0.25,58.0,X-MC,3.65,78.0,1.15,1.48,18.05,XY25,NTT,AC
35,ICICIGI,2252.93,0.58,9.55,9.71,20.19,18544.0,16656.0,190976.0,-14.13,60.0,X-MC,7.73,68.0,0.90,1.33,26.11,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,0.12,12.89,7.54,21.40,22593.0,34211.0,299637.0,4.80,63.0,X-LC,8.06,12.0,1.51,2.09,20.39,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,3.49,-34.26,71.73,12.90,103037.0,-74851.0,143645.0,111.54,64.0,H-SC,15.64,161.0,-0.73,1.00,19.44,OX40N,NTT,FINANCE
37,IEX,219.00,3.34,-3.84,54.56,48.63,104904.0,-7681.0,192273.0,-36.00,55.0,H-SC,15.28,136.0,-0.07,1.34,7.27,XR,NTT,MISC
15,CAMS,4762.00,2.20,5.23,18.59,24.79,49266.0,13160.0,265016.0,0.31,61.0,X-SC,2.25,86.0,0.27,1.85,29.57,X40N,NTT,MISC
23,DMART,5391.45,1.65,-2.74,31.72,28.12,64915.0,-5757.0,204650.0,-16.07,43.0,X-LC,5.09,19.0,-0.09,1.43,20.24,X40N,ATH,FMCG
45,JIOFIN,387.00,1.64,-0.46,25.06,24.48,66691.0,-1238.0,266127.0,-2.14,53.0,X-LC,8.71,36.0,-0.02,1.85,54.25,XY24,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-2.81,-14.49,66.27,42.18,156634.0,-40053.0,236357.0,-23.08,21.0,X-LC,0.22,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
81,VALIANTORG,838.0,-2.71,-43.69,209.28,74.15,192795.0,-71482.0,92123.0,-265.94,30.0,H-SC,13.33,149.0,-0.37,0.64,18.09,XR,NTT,CHEMICALS
80,VAIBHAVGBL,521.0,-2.55,-24.40,112.65,60.77,155660.0,-44595.0,138180.0,59.23,44.0,H-SC,11.02,159.0,-0.29,0.96,24.44,XR,NTT,JEWELLERY
56,QUESS,424.0,-2.31,-26.60,94.62,42.84,45144.0,-17295.0,47711.0,-52.07,41.0,X-SC,37.15,83.0,-0.38,0.33,3.38,XY24,NTT,MISC
58,RAJOOENG,143.1,-2.08,-25.02,86.21,39.61,66252.0,-25650.0,76850.0,-49.31,26.0,H-SC,18.73,135.0,-0.39,0.54,0.00,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.53,-2.27,112.75,107.91,162686.0,-3355.0,144289.0,-20.55,40.0,M-SC,10.13,216.0,-0.02,1.01,0.66,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.77,-7.40,8.01,0.02,12947.0,-12919.0,161640.0,146.65,77.0,M-SC,3.05,234.0,-1.00,1.13,40.60,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.50,-14.51,17.08,0.09,14715.0,-14625.0,86152.0,100.00,48.0,M-SC,4.01,253.0,-0.99,0.60,13.18,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.04,-19.94,41.74,13.47,90110.0,-53784.0,215883.0,-67.15,35.0,H-SC,1.74,158.0,-0.60,1.50,9.77,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.35,-22.67,70.72,32.03,72102.0,-29881.0,101954.0,-16.37,50.0,H-SC,6.39,148.0,-0.41,0.71,14.20,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.15,1.22,70.43,72.51,107612.0,1848.0,152793.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.32,XR,NTT,IT
38,INDIAMART,4810.62,-0.92,2.75,101.20,106.72,128244.0,3387.0,126723.0,-49.10,41.0,H-SC,4.57,139.0,0.03,0.88,26.77,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.92,2.75,101.20,106.72,128244.0,3387.0,126723.0,-49.10,41.0,H-SC,4.57,139.0,0.03,0.88,26.77,AR,ATH,MISC
85,WIPRO,420.00,0.15,1.22,70.43,72.51,107612.0,1848.0,152793.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.32,XR,NTT,IT
25,FINCABLES,1641.55,-0.53,-2.27,112.75,107.91,162686.0,-3355.0,144289.0,-20.55,40.0,M-SC,10.13,216.0,-0.02,1.01,0.66,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.55,-1.37,37.51,35.63,77538.0,-2862.0,206712.0,-15.41,37.0,H-MC,5.02,119.0,-0.04,1.44,15.08,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.84,-4.47,90.44,81.92,99180.0,-5134.0,109664.0,-44.03,33.0,M-SC,6.97,236.0,-0.05,0.76,29.82,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-2.81,-14.49,66.27,42.18,156634.0,-40053.0,236357.0,-23.08,21.0,X-LC,0.22,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.19,-34.85,107.52,35.20,90134.0,-44840.0,83830.0,9.89,23.0,X-SC,11.48,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-0.11,-18.50,67.91,36.84,70806.0,-23673.0,104265.0,-26.71,27.0,X-SC,1.45,90.0,-0.33,0.73,6.74,X40N,ATH,ELECTRICAL
46,JSWINFRA,342.00,-0.07,-6.64,23.42,15.23,43805.0,-13304.0,187043.0,-20.15,30.0,X-MC,6.16,66.0,-0.30,1.30,23.47,X40N,NTT,REALTY
54,PAGEIND,51605.08,-0.24,-5.42,33.29,26.07,51553.0,-8880.0,154860.0,-30.31,32.0,X-MC,8.44,55.0,-0.17,1.08,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-2.81,-14.49,66.27,42.18,156634.0,-40053.0,236357.0,-23.08,21.0,X-LC,0.22,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.00,0.45,-0.96,11.51,10.44,22815.0,-1922.0,198216.0,-40.74,45.0,X-LC,0.94,5.0,-0.08,1.38,4.55,X40,NTT,FMCG
36,ICICIPRULI,790.00,0.79,2.59,27.67,30.97,50168.0,4570.0,181308.0,-21.49,55.0,X-MC,1.45,75.0,0.09,1.26,15.47,X40,ATH,INSURANCE
34,HONAUT,58357.33,-0.11,-18.50,67.91,36.84,70806.0,-23673.0,104265.0,-26.71,27.0,X-SC,1.45,90.0,-0.33,0.73,6.74,X40N,ATH,ELECTRICAL
66,SIEMENS,4671.50,-1.17,-13.71,45.46,25.51,72997.0,-25520.0,160575.0,-1.39,59.0,H-LC,1.77,49.0,-0.35,1.12,18.33,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.19,-34.85,107.52,35.20,90134.0,-44840.0,83830.0,9.89,23.0,X-SC,11.48,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-0.24,-5.42,33.29,26.07,51553.0,-8880.0,154860.0,-30.31,32.0,X-MC,8.44,55.0,-0.17,1.08,0.00,X40,ATH,APPARELS
76,TMPV,600.00,-2.81,-14.49,66.27,42.18,156634.0,-40053.0,236357.0,-23.08,21.0,X-LC,0.22,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.56,-12.77,70.18,48.45,122749.0,-25600.0,174906.0,-19.01,35.0,X-MC,7.08,64.0,-0.21,1.22,0.86,X40N,ATH,FINANCE
55,PGHH,17907.65,0.67,-2.96,37.84,33.76,73743.0,-5940.0,194880.0,-31.57,38.0,X-MC,2.85,57.0,-0.08,1.36,2.09,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.31,-26.60,94.62,42.84,45144.0,-17295.0,47711.0,-52.07,41.0,X-SC,37.15,83.0,-0.38,0.33,3.38,XY24,NTT,MISC
60,RELAXO,1176.00,-0.15,-46.51,184.75,52.31,143447.0,-67516.0,77644.0,-43.08,32.0,X-SC,3.28,91.0,-0.47,0.54,3.93,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.19,-34.85,107.52,35.20,90134.0,-44840.0,83830.0,9.89,23.0,X-SC,11.48,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-0.11,-18.50,67.91,36.84,70806.0,-23673.0,104265.0,-26.71,27.0,X-SC,1.45,90.0,-0.33,0.73,6.74,X40N,ATH,ELECTRICAL
52,MEDANTA,1486.00,0.99,1.10,22.72,24.07,25311.0,1216.0,111403.0,-7.09,40.0,X-SC,4.68,89.0,0.05,0.78,20.94,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.37,-14.16,38.95,19.28,115671.0,-48982.0,296974.0,-25.24,64.0,X-LC,7.93,1.0,-0.42,2.07,9.79,X40,ATH,IT
41,INFY,2275.00,-0.07,4.84,47.73,54.87,156564.0,15132.0,328020.0,-18.52,59.0,X-LC,3.42,2.0,0.10,2.28,13.52,X40,BTT,IT
76,TMPV,600.00,-2.81,-14.49,66.27,42.18,156634.0,-40053.0,236357.0,-23.08,21.0,X-LC,0.22,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.70,-8.83,48.81,35.67,140554.0,-27881.0,287961.0,-19.67,43.0,X-LC,6.01,4.0,-0.20,2.01,3.80,X40N,ATH,FMCG
43,ITC,452.00,0.45,-0.96,11.51,10.44,22815.0,-1922.0,198216.0,-40.74,45.0,X-LC,0.94,5.0,-0.08,1.38,4.55,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,0.82,-37.83,117.57,35.27,83002.0,-42951.0,70598.0,-51.71,34.0,L-SC,19.44,270.0,-0.52,0.49,72.79,XR,NTT,HOTELS
6,ASIANTILES,137.00,-0.89,-11.71,105.77,81.67,87605.0,-10984.0,82826.0,7450.00,55.0,L-SC,18.12,271.0,-0.13,0.58,60.40,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.50,-11.86,58.32,39.54,51999.0,-11997.0,89162.0,-31.03,71.0,M-SC,6.85,220.0,-0.23,0.62,25.73,AR,ATH,AUTO
51,MASFIN,398.61,-0.10,-3.93,27.05,22.05,25461.0,-3855.0,94125.0,-17.34,55.0,H-SC,7.20,164.0,-0.15,0.66,36.82,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.60,-22.13,58.00,23.03,80878.0,-39627.0,139444.0,51.32,43.0,H-SC,12.19,167.0,-0.49,0.97,38.36,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.77,-7.40,8.01,0.02,12947.0,-12919.0,161640.0,146.65,77.0,M-SC,3.05,234.0,-1.00,1.13,40.60,OX40N,NTT,PAINTS
13,BSOFT,831.70,-0.52,-29.43,113.67,50.79,112828.0,-41394.0,99259.0,-9.55,59.0,H-SC,7.90,151.0,-0.37,0.69,15.57,XR,ATH,IT
5,ASIANPAINT,3460.25,-1.17,-0.04,20.99,20.95,52826.0,-97.0,251671.0,-1.05,75.0,X-LC,11.46,31.0,-0.00,1.75,35.58,X40,ATH,PAINTS
68,SONACOMS,806.63,0.50,-11.86,58.32,39.54,51999.0,-11997.0,89162.0,-31.03,71.0,M-SC,6.85,220.0,-0.23,0.62,25.73,AR,ATH,AUTO
0,5PAISA,593.00,3.49,-34.26,71.73,12.90,103037.0,-74851.0,143645.0,111.54,64.0,H-SC,15.64,161.0,-0.73,1.00,19.44,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.92
1,25,44.94
2,50,75.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.74
MC    29.21
LC    25.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.09
X40      20.99
X40N     11.63
XR        9.53
XY25      9.15
AR        9.13
OX40N     7.76
X200      1.89
SR        1.01
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-MC    22.72
X-LC    20.00
M-SC    12.11
X-SC     7.86
H-MC     4.77
H-LC     3.01
L-SC     1.42
M-MC     1.39
M-LC     1.06
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.86,-4.08,37.74
IT,13.68,-13.32,71.88
FINANCE,9.53,-13.38,61.28
MISC,7.52,-20.39,69.71
PAINTS,6.04,-6.40,22.21
ELECTRICAL,5.70,-10.57,49.76
INSURANCE,4.51,2.52,31.83
PHARMA,3.66,-0.72,33.68
AUTO,3.36,-19.34,70.51


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2983019.0,21
AR,1262030.0,10
XR,1261817.0,13
X40,988002.0,14
X40N,703720.0,9
OX40N,678196.0,10
XY25,327812.0,6
SR,275105.0,2
MH,66780.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3483366.0,25
M-SC,1319222.0,15
X-MC,1166985.0,16
X-LC,891172.0,11
X-SC,713434.0,8
H-MC,394598.0,3
L-SC,257094.0,3
H-LC,125652.0,2
M-LC,107612.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1188829.0      6
           AR         857176.0      5
           XR         781249.0      7
M-SC       XY24       773452.0      6
X-MC       X40        423140.0      7
X-LC       X40        390681.0      5
X-MC       XY24       337772.0      3
H-SC       OX40N      314227.0      4
M-SC       OX40N      277482.0      5
X-SC       XY24       275734.0      3
H-SC       SR         275105.0      2
X-SC       X40N       263519.0      3
X-MC       X40N       234732.0      4
X-LC       XY24       223325.0      2
H-MC       AR         210691.0      2
X-LC       X40N       205469.0      2
H-MC       XY24       183907.0      1
X-SC       X40        174181.0      2
X-MC       XY25       171341.0      2
L-SC       XR         170607.0      2
M-SC       XR         147122.0      2
           AR         121166.0      2
M-LC       XR         107612.0      1
L-SC       OX40N       86487.0      1
H-LC       AR          72997.0      1
X-LC       XY25        71697.0      2
H-SC       MH          66780.0      1
L-MC       XR          55227.0      1
H-LC       X200        52655.0      1
M-MC       XY25        45950.0      1
L-LC       XY25        38824.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 60.0 seconds
